In [13]:
#!/usr/bin/env python
from __future__ import division
import re
import os
import argparse
import numpy as np
import pandas as pd
from Bio import SeqIO
import matplotlib.pyplot as plt

RuntimeError: module compiled against API version a but this version of numpy is 9

ImportError: numpy.core.multiarray failed to import

In [1]:
def get_ctxnum(reffile):
    """
    Get the number of CG/CHG/CHH from a reference genome FASTA file
    """
    with open(reffile) as infile:
        fasta = SeqIO.to_dict(SeqIO.parse(infile, 'fasta'))
        for chr in fasta:
            fasta[chr] = str(fasta[chr].seq).upper()
    num_cg = 0
    num_chg = 0
    num_chh = 0
    for chr in fasta:
        num_cg += len([match.start() for match in re.finditer(r'(?=(CG))', fasta[chr])])
        num_cg += len([match.start()-1 for match in re.finditer(r'(?<=(CG))', fasta[chr])])
        num_chg += len([match.start() for match in re.finditer(r'(?=(C[ACT]G))', fasta[chr])])
        num_chg += len([match.start()-1 for match in re.finditer(r'(?<=(C[AGT]G))', fasta[chr])])
        num_chh += len([match.start() for match in re.finditer(r'(?=(C[ACT][ACT]))', fasta[chr])])
        num_chh += len([match.start()-1 for match in re.finditer(r'(?<=([AGT][AGT]G))', fasta[chr])])
    return num_cg, num_chg, num_chh


In [6]:
fa = "/home/jolyang/dbcenter/AGP/AGPv2/Zea_mays.AGPv2.14.dna.toplevel.fa"

In [9]:
res = get_ctxnum(fa)

In [10]:
def read_ctxcov(cgmapfile):
    """
    Read the column of coverage from CGmap file
    """
    cgmap = pd.read_table(cgmapfile, header=None, usecols=[3, 7], names=['ctx', 'cov'])
    cov_cg = cgmap[cgmap['ctx'] == 'CG']['cov']
    cov_chg = cgmap[cgmap['ctx'] == 'CHG']['cov']
    cov_chh = cgmap[cgmap['ctx'] == 'CHH']['cov']
    return cov_cg, cov_chg, cov_chh

In [11]:
res

(180125000, 158277169, 624401016)